In [14]:
import numpy as np
import pandas as pd
import re

import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [106]:
df_demographics = pd.read_csv('p2-wy-demographic-data.csv')
df_naics = pd.read_csv('p2-wy-453910-naics-data.csv')
df_population = pd.read_csv('p2-partially-parsed-wy-web-scrape.csv')
df_sales = pd.read_csv('p2-2010-pawdacity-monthly-sales.csv')

In [107]:
df_demographics = df_demographics.set_index(['City', 'County'])

In [114]:
df_naics.head(20)

,BUSINESS NAME,PHYSICAL CITY NAME,SALES VOLUME,CASS_LastLine
0,Mile High Mobile Pet LLC,Cheyenne,300000,"Cheyenne, WY 82007-3528"
1,Pets City Inc,Cheyenne,640000,"Cheyenne, WY 82009-4851"
2,Petco Animal Sups Stores Inc,Cheyenne,0,"Cheyenne, WY 82009-4945"
3,Pet-A-Care,Cheyenne,81000,"Cheyenne, WY 82009-1009"
4,Muddy Paws Pet Salon,Laramie,76000,"Laramie, WY 82070-8979"
5,Prossers Feed and Seed L L C,Wheatland,77000,"Wheatland, WY 82201-2901"
6,Pet Pals Inc of Goshen County,Torrington,126991,"Torrington, WY 82240-3516"
7,Don Bruner Sales LLC,Torrington,750000,"Torrington, WY 82240-3516"
8,Pals For Pets Inc,Saratoga,83000,"Saratoga, WY 82331"
9,Pet Barn,Worland,69000,"Worland, WY 82401-2715"


In [109]:
df_population = df_population.dropna(subset=['City|County'])
df_population['City'], df_population['County'] = df_population['City|County'].str.split('|').str
df_population['City'] = df_population['City'].apply(lambda x: x.replace('?',''))
df_population['City'] = df_population['City'].apply(lambda x: x.strip())
df_population['2014 Estimate'] = df_population['2014 Estimate'].apply(lambda x: x.split('<')[1].replace('td>','').replace(',',''))
df_population['2000 Census'] = df_population['2000 Census'].apply(lambda x: x.split('<')[1].replace('td>','').replace(',',''))
df_population['2010 Census'] = df_population['2010 Census'].apply(lambda x: x.split('<')[1].replace('td>','').replace(',',''))
df_population = df_population[['City', 'County', '2000 Census', '2010 Census', '2014 Estimate']]
df_population = df_population.set_index(['City', 'County'])

In [111]:
df_demographics.join(df_population, how='inner')

,,Land Area,Households with Under 18,Population Density,Total Families,2000 Census,2010 Census,2014 Estimate
City,County,,,,,,,
Afton,Lincoln,853.806650,497,0.93,1040.41,1818,1911,1968
Albin,Laramie,7.685902,37,0.10,74.87,120,181,185
Alpine,Lincoln,366.598892,213,0.40,446.72,550,828,845
Baggs,Carbon,253.240322,62,0.06,129.53,348,440,439
Bairoil,Sweetwater,29.459830,18,0.01,33.70,97,106,107
Bar Nunn,Natrona,208.573220,417,0.60,843.88,936,2213,2735
Basin,Big Horn,543.951304,250,0.66,566.43,1238,1285,1312
Bear River,Uinta,66.040927,98,0.33,179.24,-,518,521
Big Piney,Sublette,710.489357,186,0.31,384.80,408,552,538


In [115]:
df_sales.head(20)

,NAME,ADDRESS,CITY,STATE,ZIP,January,February,March,April,May,June,July,August,September,October,November,December
0,Pawdacity,509 Fort St # A,Buffalo,WY,82834,16200,13392,14688,17064,18360,14040,12960,19224,15984,13392,13176,16848
1,Pawdacity,601 SE Wyoming Blvd Unit 252,Casper,WY,82609,29160,21600,27000,27648,29160,27216,25488,25704,22896,25272,28944,27648
2,Pawdacity,1400 Dell Range Blvd,Cheyenne,WY,82009,47520,44280,47088,46656,43200,45144,44064,45360,47736,42984,44712,47304
3,Pawdacity,3769 E Lincolnway,Cheyenne,WY,82001,32400,26352,31968,30888,30456,32832,29808,32184,30780,31536,30024,32616
4,Pawdacity,2625 Big Horn Ave,Cody,WY,82414,19440,15984,19008,18144,16632,17496,18792,20304,19224,18144,18576,16632
5,Pawdacity,123 S 2nd St,Douglas,WY,82633,16200,13392,14688,17064,18360,14040,12960,19224,15984,29808,17496,18792
6,Pawdacity,932 Main St,Evanston,WY,82930,24840,21168,21600,22248,24192,24624,25488,25704,22032,21168,25920,24840
7,Pawdacity,900 Camel Dr,Gillette,WY,82716,23760,20952,24192,25056,21168,20520,20304,24624,24192,23760,22032,22464
8,Pawdacity,200 E Lakeway Rd,Gillette,WY,82718,23760,20844,24192,22032,21168,21384,21816,22464,24840,24408,20952,22248
9,Pawdacity,180 S Bent St,Powell,WY,82435,20520,17928,20304,21168,21600,17928,18144,18576,20304,21168,17496,18792
